In [1]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm

# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal

# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)

from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType

from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm

In [2]:
rospy.init_node('emergency_lamp_demo')

In [3]:
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
#pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)

def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()

def cancel(cl):
    cl.cancel_all_goals()
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)

### Load robot

In [4]:
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")
#cobot.change_tool("nothing")

pybullet build time: May  8 2021 05:48:13


[INFO]some functions are not yet compiled. Expect delays upon first call


In [5]:
!rosparam get /hrr_cobot/tool_name

nothing


cobot.change_tool("shaftgrinder")
cobot.tool_controller.run_shaftgrinder(1.0,"slow")

### Initiate all Clients

In [6]:
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)

# Emergency Lamp disassembly

## Tool Change to Shaftgrinder

In [7]:
cobot.close_tool_changer()

In [8]:
cobot.open_tool_changer()

In [ ]:
cobot.open_tool_changer()
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.SHAFT_GRINDER
tool_change_client.send_goal(tc_goal)


In [ ]:
tool_change_client.cancel_all_goals()

## Cutting (6 goals)
cutting.yaml

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.tool

### Clip 1

In [ ]:
material= MaterialType()
material = material.PLASTIC

normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.44808836914,0.352939522502,0.055], "pointA")#[0.6921,0.2472,0.], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.447987593911,0.363652997478,0.055], "pointB")
## replace by vision data

In [ ]:
goal=CuttingGoal()
goal.surface_normal_start=normalA
goal.surface_normal_end=normalB
goal.start_location=pointA
goal.end_location=pointB
goal.material.material_type=material
goal.timeout=5000
goal.thickness = 0.005 #not important. can be anything

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cobot.tool_controller.run_shaftgrinder(2, "slow")

# Hide cobot

In [ ]:
joi = cobot.q_calib
joi[0]=-np.deg2rad(90)
cobot.move_to_joint_pose(joi, stochastic=True)

In [15]:
!rosparam get /hrr_cobot/tool_name

nothing


In [14]:
print(cobot)

Hrr-Cobot state:
FT-data:
current force:	[ 2.36481 -1.13599 -4.8252 ] [N]
current torque:	[1.24939 0.6665  1.96826] [Nm]
current wrench:	[ 2.37994 -1.1299  -4.91521  1.27282  0.65967  1.94496] [N,Nm]
=>in contact:	True
---
Robot-data:
q:		[-77.12682  37.25112 -76.98605  -0.04873  69.72958  89.73867][°]
q_dot:		[0. 0. 0. 0. 0. 0.][rad/s]
tau:		[0. 0. 0. 0. 0. 0.][Nm]
pos:		[0.14272 0.62478 0.46687][m]
quat:		0.03436 <[ 0.99282 -0.11446 -0.00419]>
tool-pos:	[0.14272 0.62478 0.46687][m]
tool-rpy:	[ 13.12173  -0.92764 176.14293][°]
robot-status:	ready
---


## Tool Change to Vacuum

In [ ]:
#Right now we are on the left side of the robot, should be save this way

In [16]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

In [17]:
#From hide cobot
joi = cobot.q_calib.copy()
joi[0]=np.deg2rad(80) #good
cobot.move_to_joint_pose(joi, stochastic=False)

In [18]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.VACUUM_GRIPPER
tool_change_client.send_goal(tc_goal)

In [ ]:
tool_change_client.cancel_all_goals()

## Vacuum Top/  Middle Cover

In [ ]:
!rosparam get /hrr_cobot/tool_name

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
cobot._tool_controller.vacuum = False

In [ ]:
      x: 0.516095816448
      y: 0.0741897862287
      z: 0.0249991445966

In [23]:
obj_pos_guess = np.r_[0.516095816448 ,0.074189, 0.024999] #[0.66, 0.17, 0.07] is actual vision data for elamp cover
obj_orient_guess = np.quaternion(0,-0.6495, 0.029, 0.7597) #actual vision data (dont care for now)

release_pos = np.r_[1, 0.1, 0.1] # will drop the object here, careful!
release_orientation = np.quaternion(0,-0.715468876127, 0.0542776712358, 0.696533001155) #dont care for now

In [24]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

# vac_goal.timeout = 120.0
# vac_goal

In [25]:
vacuum_client.send_goal(vac_goal)

### In between top and middle cover, so vision can identify

In [22]:
joi = cobot.q_calib.copy()
joi[0]=np.deg2rad(-80) #good
cobot.move_to_joint_pose(joi, stochastic=False)

## Tool Change to Pneumatic Gripper

In [ ]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)
#cobot.move_to_joint_pose(np.r_[ 0,  np.deg2rad(17), np.deg2rad(-75),  0.00081,  1.6057 , -1.80819], stochastic=False)
#cobot.move_to_joint_pose(np.r_[ 2.29384,  np.deg2rad(17), np.deg2rad(-75),  0.00081,  1.6057 , -1.80819], stochastic=False)

In [26]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50
tool_change_client.send_goal(tc_goal)

## Remove Battery
fingergrasp

In [27]:
cobot.move_to_joint_pose(np.r_[ 2.29384,  np.deg2rad(17), np.deg2rad(-75),  0.00081,  1.6057 , -1.80819], stochastic=False)

### Wait here for detection of battery

In [28]:
cobot.move_to_joint_pose(np.r_[ 0,  np.deg2rad(17), np.deg2rad(-75),  0.00081,  1.6057 , -1.80819], stochastic=False)
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

In [ ]:
     x: 0.562424537926
      y: 0.0664328784315
      z: 0.0183034314412

In [29]:
obj_pos_guess = np.r_[0.562424537926,0.0664328784315 ,0.0183034314412 ] #[0.712, 0.263, 0.0196] is actual vision data for battery
obj_orient_guess = np.quaternion(0.0,-0.4376,0.01394,0.8990) #actual vision data for battery
release_pos = np.r_[0.5, 0.1, 0.3] #choose so it makes sense
release_orientation = np.quaternion(0.0,-0.657320580033,-0.0241373269709,0.753224431702) #same orientaiton for ease

In [30]:
pgrip_goal = FingerGraspGoal()
pgrip_goal.object_center.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
pgrip_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

pgrip_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
pgrip_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

pgrip_goal.timeout = 10000.0
#pgrip_goal.gripper_open_width = 0.084
#pgrip_goal.gripper_close_width = 0

In [35]:
pgrip_client.send_goal(pgrip_goal)

In [34]:
pgrip_client.cancel_goals()

AttributeError: 'SimpleActionClient' object has no attribute 'cancel_goals'

## Move to pneumatic position

In [ ]:
cobot.move_to_joint_pose(np.r_[2.29483,  np.deg2rad(17), np.deg2rad(-75),  0.00081,  1.6057 , -1.80819], stochastic=False)

## Tool Change to None

In [ ]:
tc_goal = ChangeToolGoal()
tc_goal.new_tool.type = tc_goal.new_tool.NONE
tool_change_client.send_goal(tc_goal)